# Enrichissement des données
Ce notebook sert à l'enrichissement des données, en se basant sur les coordonnées GPS fournies. Les données ajoutées proviennent des différents jeux de données disponibles sur GeoAdmin, ainsi que de Google Earth Engine, et qui ont été prétraités dans un autre notebook. 

## Import des données de base
Afin de pouvoir enrichir les données, il nous faut dans un premier temps importer le contenu du fichier contenant les données de présence de l'espèce invasive sélectionnée.

In [1]:
# Imports nécessaires pour ce notebook
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point

In [2]:
berce_file_path = '../../data/processed/df_berce_clean.csv'

In [3]:
# Importation des données depuis le fichier csv
df_berce = pd.read_csv(berce_file_path, index_col=0)

# Affichage pour vérifier l'importation des données
df_berce.head()

,decimalLatitude,decimalLongitude,elevation,date
0,46.29982,7.91708,658.0,1899-12-30 00:00:00
1,47.14918,7.50258,500.0,1905-06-29 00:00:00
2,47.14918,7.50258,500.0,1905-06-28 00:00:00
14,46.39338,6.99140,1448.0,1970-01-01 00:00:00
15,46.34986,7.16844,1246.5,1970-01-01 00:00:00


In [4]:
swiss_data_file_path = '../../data/processed/processed_enhanced_swiss_data.csv'

In [5]:
# Importation des données depuis le fichier csv
df_swiss = pd.read_csv(swiss_data_file_path, index_col=0)

# Applications de paramètres liés à la géométrie du GeoDataFrame
df_swiss['geometry'] = df_swiss['geometry'].apply(wkt.loads)
gdf_swiss = gpd.GeoDataFrame(df_swiss, geometry='geometry')
gdf_swiss.set_crs(epsg=2056, inplace=True)

# Affichage pour vérifier l'importation des données
gdf_swiss.head()

,geometry,altitude,TypoCH_NUM,NAHRSTOFF,WASSERDURC,WASSERSPEI,temperature,Bodentyp_Rendzina(Jura),Bodentyp_Ranker,Bodentyp_Rendzina,Bodentyp_Luvisol,Bodentyp_Cambisol_(feinkörnige_Variante),Bodentyp_Gleysol,Bodentyp_Podzol,Bodentyp_Fluvisol,Bodentyp_Regosol,Bodentyp_Cambisol,Bodentyp_Lithosol,Bodentyp_Histosol
idx,,,,,,,,,,,,,,,,,,,
35,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...",363.5148,62.0,4.0,6.0,4.0,5.2,-,-,-,orthic,-,-,-,-,-,"eutric, calcaric",-,-
36,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...",358.9348,62.0,4.0,6.0,4.0,5.2,-,-,-,orthic,-,-,-,-,-,"eutric, calcaric",-,-
261,"POLYGON ((2486071.58 1109261.72, 2487071.58 11...",415.6684,62.0,4.0,4.0,4.0,5.3,-,-,-,-,-,-,-,-,eutric,spododystric,-,-
262,"POLYGON ((2486071.58 1110261.72, 2487071.58 11...",368.5936,62.0,4.0,4.0,4.0,5.3,-,-,-,-,-,-,-,-,eutric,spododystric,-,-
263,"POLYGON ((2486071.58 1111261.72, 2487071.58 11...",352.9472,82.0,4.0,6.0,4.0,5.3,-,-,-,orthic,-,-,-,-,-,"eutric, calcaric",-,-


## Jointure des jeux de données

Maintenant que les deux jeux sont importés, nous pouvons les combiner. Pour cela, deux nouvelles colonnes vont être créées : `presence` et `count`.
La première caractéristique va permettre d'identifier facilement les cellules dans lesquelles une plante a été détectée, tandis que la seconde permettra de savoir combien de plant ont été observés. Cela nous permettra, si besoin, d'utiliser cette information lors de nos prédictions.

La première étape consiste à modifier le jeu de données contenant notre néophyte, afin que celui-ci soit un geodataframe et que ses coordonnées soient dans le bon format pour être traitées. En effet, les coordonnées utilisées dans ces données sont des coordonnées géographiques, incompatibles avec les données d'enrichissement. Il est donc nécessaire de les transformer dans un format qui pourra être exploité plus facilement. Pour cela, le format EPSG:2056 ([LV95](https://fr.wikipedia.org/wiki/Syst%C3%A8me_de_coordonn%C3%A9es_g%C3%A9ographiques_suisse#MN95)) va être utilisé. Pour rappel, il s'agit d'un système de coordonnées [géodésiques](https://fr.wikipedia.org/wiki/G%C3%A9od%C3%A9sique) utilisé principalement en Suisse et format par défaut pour une grande partie des services offerts par l'API de GeoAdmin et utilisé dans nos données d'enrichissement.

In [6]:
# Tranformation en geodataframe
geometry = [Point(xy) for xy in zip(df_berce['decimalLongitude'], df_berce['decimalLatitude'])]
gdf_berce = gpd.GeoDataFrame(df_berce, geometry=geometry, crs="EPSG:4326")

# Conversion des coordonnées géographiques en LV95
gdf_berce = gdf_berce.to_crs(epsg=2056)

gdf_berce.head()

,decimalLatitude,decimalLongitude,elevation,date,geometry
0,46.29982,7.91708,658.0,1899-12-30 00:00:00,POINT (2636863.004 1127713.248)
1,47.14918,7.50258,500.0,1905-06-29 00:00:00,POINT (2604850.377 1222024.964)
2,47.14918,7.50258,500.0,1905-06-28 00:00:00,POINT (2604850.377 1222024.964)
14,46.39338,6.99140,1448.0,1970-01-01 00:00:00,POINT (2565599.774 1138100.346)
15,46.34986,7.16844,1246.5,1970-01-01 00:00:00,POINT (2579200.238 1133199.999)


Nous pouvons maintenant ajouter les nouvelles colonnes à notre geodataframe contenant les données environnementales de la Suisse et les remplir avec les données associées.

Note : Cette étape peut prendre un peu de temps, au vu du nombre de cellules à vérifier.

In [7]:
# Création des nouvelles colonnes, initialisées à 0
gdf_swiss['Presence'] = 0
gdf_swiss['Count'] = 0

# Comptage des points présents dans chaque zone
for i, cellule in gdf_swiss.iterrows():
    # Compter les points de gdf_berce qui sont dans la zone actuelle
    points_in_zone = gdf_berce[gdf_berce.geometry.within(cellule.geometry)]
    # Modification des nouvelles colonnes en cas de présence de la plante invasive
    if len(points_in_zone) > 0 :
        gdf_swiss.at[i, 'Presence'] = 1
        gdf_swiss.at[i, 'Count'] += len(points_in_zone)

gdf_swiss.head()

,geometry,altitude,TypoCH_NUM,NAHRSTOFF,WASSERDURC,WASSERSPEI,temperature,Bodentyp_Rendzina(Jura),Bodentyp_Ranker,Bodentyp_Rendzina,...,Bodentyp_Cambisol_(feinkörnige_Variante),Bodentyp_Gleysol,Bodentyp_Podzol,Bodentyp_Fluvisol,Bodentyp_Regosol,Bodentyp_Cambisol,Bodentyp_Lithosol,Bodentyp_Histosol,Presence,Count
idx,,,,,,,,,,,,,,,,,,,,,
35,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...",363.5148,62.0,4.0,6.0,4.0,5.2,-,-,-,...,-,-,-,-,-,"eutric, calcaric",-,-,0,0
36,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...",358.9348,62.0,4.0,6.0,4.0,5.2,-,-,-,...,-,-,-,-,-,"eutric, calcaric",-,-,0,0
261,"POLYGON ((2486071.58 1109261.72, 2487071.58 11...",415.6684,62.0,4.0,4.0,4.0,5.3,-,-,-,...,-,-,-,-,eutric,spododystric,-,-,0,0
262,"POLYGON ((2486071.58 1110261.72, 2487071.58 11...",368.5936,62.0,4.0,4.0,4.0,5.3,-,-,-,...,-,-,-,-,eutric,spododystric,-,-,0,0
263,"POLYGON ((2486071.58 1111261.72, 2487071.58 11...",352.9472,82.0,4.0,6.0,4.0,5.3,-,-,-,...,-,-,-,-,-,"eutric, calcaric",-,-,0,0


## Sauvegarde des données
Afin de ne pas devoir effectuer le traitement précédent dans chaque notebook, le geodataframe doit être enregistré.

In [8]:
# Sauvegarde du DataFrame dans un fichier CSV
filename = '../../data/processed/processed_enhanced_data_with_presence.csv'
gdf_swiss.to_csv(filename, index=True, encoding='utf-8')

print(f"Le DataFrame a été sauvegardé dans à l'emplacement '{filename}'")

Le DataFrame a été sauvegardé dans à l'emplacement '../../data/processed/processed_enhanced_data_with_presence.csv'
